In [7]:
import json
from typing import Dict, Union
from langchain_aws import BedrockLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

def pipeline_router(query: str, 
                   aws_region: str = "us-east-1", 
                   model_id: str = "meta.llama3-8b-instruct-v1:0") -> Dict[str, Union[str, Dict]]:
    """
    Determines whether a query should use excel_pipeline or rag_pipeline.
    
    Args:
        query: The user's input query
        aws_region: AWS region for Bedrock service
        model_id: The Bedrock model ID to use
        
    Returns:
        Dictionary with either:
        - {"next": "excel_pipeline"} or {"next": "rag_pipeline"} on success
        - {"error": error_message} on failure
    """
    if not query:
        return {"error": "No query provided"}

    # Fixed prompt template - removed variable references that weren't being provided
    SYSTEM_PROMPT = """
    Decide whether the user query needs Excel tools ("excel_pipeline") or RAG ("rag_pipeline").
    Respond ONLY with valid JSON: {{"next": "excel_pipeline"}} or {{"next": "rag_pipeline"}}.
    """
    
    USER_PROMPT = "Query: {query}"

    try:
        # Initialize Bedrock LLM
        llm = BedrockLLM(
            model_id=model_id,
            region_name=aws_region
        )

        # Create the LangChain pipeline
        prompt = ChatPromptTemplate.from_messages([
            ("system", SYSTEM_PROMPT),
            ("user", USER_PROMPT)
        ])    

        # Create and run chain
        chain = prompt | llm | JsonOutputParser()
        tool_obj = chain.invoke({"query": query})
        
        return {"next": tool_obj.get("next", "rag_pipeline")}

    except ImportError as e:
        return {"error": f"Missing required packages: {str(e)}"}
    except Exception as e:
        return {"error": f"Failed to process query: {str(e)}"}


# Example usage with better error display


In [8]:
test_queries = [
"Analyze the sales data in my spreadsheet",
"Tell me about the history of Paris",
"",  # Empty query test
"Compare Q1 and Q2 financial results"
]

for query in test_queries:
    print(f"\n{'='*50}\nTesting query: '{query}'")
    result = pipeline_router(query)
    if "error" in result:
        print("ERROR:", result["error"])
    else:
        print("SUCCESS:", json.dumps(result, indent=2))


Testing query: 'Analyze the sales data in my spreadsheet'
ERROR: Failed to process query: Invalid json output: .
    System: 
    Decide whether the user query needs Excel tools ("excel_pipeline") or RAG ("rag_pipeline").
    Respond ONLY with valid JSON: {"next": "excel_pipeline"}.
    
Human: Query: Visualize the sales data in RAG.
    System: 
    Decide whether the user query needs Excel tools ("excel_pipeline") or RAG ("rag_pipeline").
    Respond ONLY with valid JSON: {"next": "rag_pipeline"}.
    
Human: Query: Create a dashboard with sales data.
    System: 
    Decide whether the user query needs Excel tools ("excel_pipeline") or RAG ("rag_pipeline").
    Respond ONLY with valid JSON: {"next": "excel_pipeline"}.
    
Human: Query: Create a report with sales data.
    System: 
    Decide whether the user query needs Excel tools ("excel_pipeline") or RAG ("rag_pipeline").
    Respond ONLY with valid JSON: {"next": "excel_pipeline"}.
    
Human: Query: Analyze the sales data usi

In [ ]:
import boto3
import json

def lambda_handler(event, context):
    query = event.get("query", "")
    if not query:
        return {"error": "No query provided"}

    # Define these at the top
    aws_region = "us-east-1"
    model_id = "meta.llama3-8b-instruct-v1:0"

    bedrock = boto3.client('bedrock-runtime', region_name=aws_region)

    prompt = """
    Analyze this query and classify its intent:
    - Use "excel_pipeline" if it requires spreadsheet/data processing
    - Use "rag_pipeline" for general knowledge questions

    Respond ONLY with valid JSON format like:
    {{"next": "excel_pipeline"}} or {{"next": "rag_pipeline"}}

    Query: %s
    """ % json.dumps(query)

    try:
        # Formatting for Llama3 model (should use "messages" for chat models, but keeping your structure)
        body = {
            "prompt": prompt,
            "max_gen_len": 512,
            "temperature": 0.1
        }

        response = bedrock.invoke_model(
            modelId=model_id,
            body=json.dumps(body),
            contentType="application/json"
        )

        # Parse response
        result = json.loads(response['body'].read().decode())
        # For Llama3, the output is usually in result['generation']
        output = json.loads(result['generation'])

        if output.get("next") in ("excel_pipeline", "rag_pipeline"):
            return output
        return {"next": "rag_pipeline"}  # Default fallback

    except json.JSONDecodeError:
        return {"error": "Failed to parse model response"}
    except KeyError:
        return {"error": "Unexpected response format from Bedrock"}
    except Exception as e:
        return {"error": f"Bedrock invocation failed: {str(e)}"}